In [24]:
import pandas as pd

In [25]:
import requests

In [27]:
import json

In [2]:
!pip install python-dotenv


In [14]:
import requests
import time
import pandas as pd
from datetime import datetime, timedelta

# configuration
API_KEY = "Faf6cf1b012b49b78aa1131a5069a8fd"  #  football-data.org API key
BASE_URL = "https://api.football-data.org/v4"
HEADERS = {"X-Auth-Token": API_KEY}

# Top 5 Leagues
LEAGUES = {
    "PL": "Premier League",
    "PD": "La Liga",
    "SA": "Serie A",
    "BL1": "Bundesliga",
    "FL1": "Ligue 1"
}



def call_api(url):
    """
    Handles API requests with retry on rate limits.
    """
    while True:
        res = requests.get(url, headers=HEADERS)

        if res.status_code == 429:  # Rate limit hit
            print("⚠️ Rate limit hit. Waiting 30 seconds...")
            time.sleep(30)
            continue
        elif res.status_code == 200:
            return res.json()
        else:
            res.raise_for_status()

def fetch_recent_matches(comp_code, days_back=7):
    """
    Fetch recent matches for a given competition.
    """
    today = datetime.today()
    date_from = (today - timedelta(days=days_back)).strftime("%Y-%m-%d")
    date_to = today.strftime("%Y-%m-%d")

    url = f"{BASE_URL}/matches?competitions={comp_code}&dateFrom={date_from}&dateTo={date_to}"
    data = call_api(url)
    return data.get("matches", [])

def fetch_team_squad(team_id):
    """
    Fetch squad (players) for a given team.
    """
    url = f"{BASE_URL}/teams/{team_id}"
    data = call_api(url)
    return data.get("squad", [])


# MAIN ETL FUNCTION


def run_etl():
    all_matches = []
    all_players = []

    print("🚀 Starting Football ETL Pipeline...")

    for code, name in LEAGUES.items():
        print(f"\n📌 Fetching matches for {name} ({code})...")
        matches = fetch_recent_matches(code, days_back=7)

        print(f"   ➡ Found {len(matches)} matches")
        for match in matches:
            # Extract match data
            match_info = {
                "Match ID": match["id"],
                "Competition": name,
                "Date": match["utcDate"],
                "Home Team": match["homeTeam"]["name"],
                "Away Team": match["awayTeam"]["name"],
                "Score Home": match["score"]["fullTime"]["home"],
                "Score Away": match["score"]["fullTime"]["away"],
                "Status": match["status"]
            }
            all_matches.append(match_info)

            # Fetch squads for home & away teams
            for team in [match["homeTeam"], match["awayTeam"]]:
                squad = fetch_team_squad(team["id"])
                for player in squad:
                    all_players.append({
                        "Player ID": player["id"],
                        "Player Name": player["name"],
                        "Team": team["name"],
                        "Position": player.get("position", "N/A"),
                        "Nationality": player.get("nationality", "N/A"),
                        "DOB": player.get("dateOfBirth", "N/A")
                    })
                time.sleep(6)  # Respect API limit

    # Convert to DataFrames
    df_matches = pd.DataFrame(all_matches)
    df_players = pd.DataFrame(all_players)

    # Save to CSV for PostgreSQL import
    df_matches.to_csv("matches.csv", index=False)
    df_players.to_csv("players.csv", index=False)

    print("\n✅ ETL Completed Successfully!")
    print(f"   ➡ Matches saved: {len(df_matches)}")
    print(f"   ➡ Players saved: {len(df_players)}")

    return df_matches, df_players


# RUN SCRIPT

if __name__ == "__main__":
    matches_df, players_df = run_etl()


🚀 Starting Football ETL Pipeline...

📌 Fetching matches for Premier League (PL)...
   ➡ Found 10 matches

📌 Fetching matches for La Liga (PD)...
   ➡ Found 10 matches

📌 Fetching matches for Serie A (SA)...
   ➡ Found 8 matches

📌 Fetching matches for Bundesliga (BL1)...
   ➡ Found 9 matches

📌 Fetching matches for Ligue 1 (FL1)...
   ➡ Found 9 matches

✅ ETL Completed Successfully!
   ➡ Matches saved: 46
   ➡ Players saved: 2822


In [19]:
import os

print(os.getcwd())      # See current working directory
print(os.listdir())     # List all files here



C:\Users\user\Hagital Data Engineering Boothcamp
['.ipynb_checkpoints', 'accessment raw_rentals.ipynb', 'assignment 7.ipynb', 'Classwork.ipynb', 'cleaned_accessment', 'cleaned_data.csv', 'Football Match Performance Analytics(Sport).ipynb', 'Group 3 Project.ipynb', 'la_liga_2025_matches.csv', 'la_liga_2025_players.csv', 'leagues.json', 'lesson 8 and 9 .ipynb', 'matches.csv', 'matches.json', 'matches_stats.csv', 'match_stats', 'multi_league_matches.csv', 'multi_league_players.csv', 'netflix.ipynb', 'netflix_cleaned.ipynb', 'netflix_titles_cleaned.csv', 'nigeria_stock_cleaned_data.csv', 'players.csv', 'players.json', 'players_cache.json', 'player_stats', 'player_stats.json', 'premier_league_2023_matches.csv', 'premier_league_2023_players.csv', 'premier_teams_cache.json', 'recent_matches.csv', 'teams.json', 'teams_cache.json', 'titanic assignment.ipynb', 'Titanic.ipynb', 'titanic_cleaned_data.csv', 'titanic_new.ipynb', 'Trend & Risk Analysis of the Nigerian Stock Market (2008–2020.ipynb']


In [20]:
import pandas as pd

# Load the correct files
matches_df = pd.read_csv("matches.csv")
players_df = pd.read_csv("players.csv")

# Check shapes and preview
print(f"Matches: {matches_df.shape} | Players: {players_df.shape}")
print("\nMatches Data:")
display(matches_df.head(10))

print("\nPlayers Data:")
display(players_df.head(10))


Matches: (46, 8) | Players: (2822, 6)

Matches Data:


,Match ID,Competition,Date,Home Team,Away Team,Score Home,Score Away,Status
0,537794,Premier League,2025-08-18T19:00:00Z,Leeds United FC,Everton FC,1,0,FINISHED
1,537804,Premier League,2025-08-22T19:00:00Z,West Ham United FC,Chelsea FC,1,5,FINISHED
2,537802,Premier League,2025-08-23T11:30:00Z,Manchester City FC,Tottenham Hotspur FC,0,2,FINISHED
3,537798,Premier League,2025-08-23T14:00:00Z,Brentford FC,Aston Villa FC,1,0,FINISHED
4,537799,Premier League,2025-08-23T14:00:00Z,Burnley FC,Sunderland AFC,2,0,FINISHED
5,537795,Premier League,2025-08-23T14:00:00Z,AFC Bournemouth,Wolverhampton Wanderers FC,1,0,FINISHED
6,537797,Premier League,2025-08-23T16:30:00Z,Arsenal FC,Leeds United FC,5,0,FINISHED
7,537800,Premier League,2025-08-24T13:00:00Z,Everton FC,Brighton & Hove Albion FC,2,0,FINISHED
8,537796,Premier League,2025-08-24T13:00:00Z,Crystal Palace FC,Nottingham Forest FC,1,1,FINISHED
9,537801,Premier League,2025-08-24T15:30:00Z,Fulham FC,Manchester United FC,1,1,FINISHED



Players Data:


,Player ID,Player Name,Team,Position,Nationality,DOB
0,1331,Lucas Perri,Leeds United FC,Goalkeeper,Brazil,1997-12-10
1,5315,Alex Cairns,Leeds United FC,Goalkeeper,England,1993-01-04
2,7913,Karl Darlow,Leeds United FC,Goalkeeper,England,1990-10-08
3,74831,Illan Meslier,Leeds United FC,Goalkeeper,France,2000-03-02
4,204504,Darko Gyabi,Leeds United FC,Midfield,England,2004-02-18
5,265196,Sam Chambers,Leeds United FC,Midfield,Scotland,2007-08-18
6,180813,Wilfried Gnonto,Leeds United FC,Right Winger,Italy,2003-11-05
7,157406,Sam Greenwood,Leeds United FC,Attacking Midfield,England,2002-01-26
8,136349,Largie Ramazani,Leeds United FC,Left Winger,Belgium,2001-02-27
9,122231,Isaac Schmidt,Leeds United FC,Left-Back,Switzerland,1999-12-07


In [21]:
matches_df.shape

(46, 8)

In [22]:
players_df.shape

(2822, 6)

In [23]:
matches_df.isnull().sum()

Match ID       0
Competition    0
Date           0
Home Team      0
Away Team      0
Score Home     0
Score Away     0
Status         0
dtype: int64

In [24]:
players_df.isnull().sum()

Player ID      0
Player Name    0
Team           0
Position       0
Nationality    0
DOB            4
dtype: int64

In [25]:
matches_df.columns.tolist()

['Match ID',
 'Competition',
 'Date',
 'Home Team',
 'Away Team',
 'Score Home',
 'Score Away',
 'Status']

In [26]:
players_df.columns.tolist()

['Player ID', 'Player Name', 'Team', 'Position', 'Nationality', 'DOB']

In [35]:
players_df.dtypes

Player ID               int64
Player Name            object
Team                   object
Position               object
Nationality            object
DOB            datetime64[ns]
Age                     int64
dtype: object

In [36]:
matches_df.dtypes

Match ID                           int64
Competition                       object
Date                 datetime64[ns, UTC]
Home Team                         object
Away Team                         object
Score Home                         int64
Score Away                         int64
Status                            object
Total Goals                        int64
Result                            object
Home Possession %                float64
Away Possession %                float64
dtype: object

In [28]:
# Convert "Date" to datetime format
matches_df["Date"] = pd.to_datetime(matches_df["Date"], errors="coerce")

# Calculate total goals
matches_df["Total Goals"] = matches_df["Score Home"] + matches_df["Score Away"]

# Determine match result
matches_df["Result"] = matches_df.apply(
    lambda x: "Home Win" if x["Score Home"] > x["Score Away"]
    else ("Away Win" if x["Score Away"] > x["Score Home"] else "Draw"),
    axis=1
)

display(matches_df.head(5))


,Match ID,Competition,Date,Home Team,Away Team,Score Home,Score Away,Status,Total Goals,Result
0,537794,Premier League,2025-08-18 19:00:00+00:00,Leeds United FC,Everton FC,1,0,FINISHED,1,Home Win
1,537804,Premier League,2025-08-22 19:00:00+00:00,West Ham United FC,Chelsea FC,1,5,FINISHED,6,Away Win
2,537802,Premier League,2025-08-23 11:30:00+00:00,Manchester City FC,Tottenham Hotspur FC,0,2,FINISHED,2,Away Win
3,537798,Premier League,2025-08-23 14:00:00+00:00,Brentford FC,Aston Villa FC,1,0,FINISHED,1,Home Win
4,537799,Premier League,2025-08-23 14:00:00+00:00,Burnley FC,Sunderland AFC,2,0,FINISHED,2,Home Win


In [29]:
# Define a function to calculate home & away possession %
def calc_possession(home, away):
    if home + away == 0:
        return 50, 50
    home_possession = round((home / (home + away)) * 100, 2)
    away_possession = round((away / (home + away)) * 100, 2)
    return home_possession, away_possession

# Apply to the dataset
matches_df[["Home Possession %", "Away Possession %"]] = matches_df.apply(
    lambda x: pd.Series(calc_possession(x["Score Home"], x["Score Away"])),
    axis=1
)

display(matches_df.head(5))


,Match ID,Competition,Date,Home Team,Away Team,Score Home,Score Away,Status,Total Goals,Result,Home Possession %,Away Possession %
0,537794,Premier League,2025-08-18 19:00:00+00:00,Leeds United FC,Everton FC,1,0,FINISHED,1,Home Win,100.00,0.00
1,537804,Premier League,2025-08-22 19:00:00+00:00,West Ham United FC,Chelsea FC,1,5,FINISHED,6,Away Win,16.67,83.33
2,537802,Premier League,2025-08-23 11:30:00+00:00,Manchester City FC,Tottenham Hotspur FC,0,2,FINISHED,2,Away Win,0.00,100.00
3,537798,Premier League,2025-08-23 14:00:00+00:00,Brentford FC,Aston Villa FC,1,0,FINISHED,1,Home Win,100.00,0.00
4,537799,Premier League,2025-08-23 14:00:00+00:00,Burnley FC,Sunderland AFC,2,0,FINISHED,2,Home Win,100.00,0.00


In [30]:
# Convert DOB to datetime & handle missing values
players_df["DOB"] = pd.to_datetime(players_df["DOB"], errors="coerce")
players_df["DOB"].fillna(players_df["DOB"].median(), inplace=True)

# Calculate player age
today = pd.Timestamp.today()
players_df["Age"] = players_df["DOB"].apply(lambda x: today.year - x.year if pd.notnull(x) else None)

display(players_df.head(5))


,Player ID,Player Name,Team,Position,Nationality,DOB,Age
0,1331,Lucas Perri,Leeds United FC,Goalkeeper,Brazil,1997-12-10,28
1,5315,Alex Cairns,Leeds United FC,Goalkeeper,England,1993-01-04,32
2,7913,Karl Darlow,Leeds United FC,Goalkeeper,England,1990-10-08,35
3,74831,Illan Meslier,Leeds United FC,Goalkeeper,France,2000-03-02,25
4,204504,Darko Gyabi,Leeds United FC,Midfield,England,2004-02-18,21


In [31]:
players_df.isnull().sum()

Player ID      0
Player Name    0
Team           0
Position       0
Nationality    0
DOB            0
Age            0
dtype: int64

In [37]:
import numpy as np

# Simulate passes attempted and completed
np.random.seed(42)
players_df["Passes Attempted"] = np.random.randint(20, 80, size=len(players_df))
players_df["Passes Completed"] = np.random.randint(10, 75, size=len(players_df))

# Calculate pass accuracy
players_df["Pass Accuracy %"] = round(
    (players_df["Passes Completed"] / players_df["Passes Attempted"]) * 100, 2
)

display(players_df.head(5))


,Player ID,Player Name,Team,Position,Nationality,DOB,Age,Passes Attempted,Passes Completed,Pass Accuracy %
0,1331,Lucas Perri,Leeds United FC,Goalkeeper,Brazil,1997-12-10,28,58,57,98.28
1,5315,Alex Cairns,Leeds United FC,Goalkeeper,England,1993-01-04,32,71,55,77.46
2,7913,Karl Darlow,Leeds United FC,Goalkeeper,England,1990-10-08,35,48,52,108.33
3,74831,Illan Meslier,Leeds United FC,Goalkeeper,France,2000-03-02,25,34,30,88.24
4,204504,Darko Gyabi,Leeds United FC,Midfield,England,2004-02-18,21,62,32,51.61


In [38]:
# Get number of wins per team
team_wins = matches_df.groupby("Home Team")["Result"].apply(lambda x: (x=="Home Win").sum()).to_dict()

# Calculate player performance score
players_df["Performance Score"] = players_df.apply(
    lambda x: (team_wins.get(x["Team"], 0) * 5) + (x["Pass Accuracy %"] * 0.3),
    axis=1
)

display(players_df.head(5))


,Player ID,Player Name,Team,Position,Nationality,DOB,Age,Passes Attempted,Passes Completed,Pass Accuracy %,Performance Score
0,1331,Lucas Perri,Leeds United FC,Goalkeeper,Brazil,1997-12-10,28,58,57,98.28,34.484
1,5315,Alex Cairns,Leeds United FC,Goalkeeper,England,1993-01-04,32,71,55,77.46,28.238
2,7913,Karl Darlow,Leeds United FC,Goalkeeper,England,1990-10-08,35,48,52,108.33,37.499
3,74831,Illan Meslier,Leeds United FC,Goalkeeper,France,2000-03-02,25,34,30,88.24,31.472
4,204504,Darko Gyabi,Leeds United FC,Midfield,England,2004-02-18,21,62,32,51.61,20.483


In [39]:
matches_df.to_csv("transformed_matches.csv", index=False)
players_df.to_csv("transformed_players.csv", index=False)

print("✅ Transformation Completed Successfully!")
print(f"Transformed matches saved: {len(matches_df)} rows")
print(f"Transformed players saved: {len(players_df)} rows")


✅ Transformation Completed Successfully!
Transformed matches saved: 46 rows
Transformed players saved: 2822 rows


In [41]:
matches_df.head()

,Match ID,Competition,Date,Home Team,Away Team,Score Home,Score Away,Status,Total Goals,Result,Home Possession %,Away Possession %
0,537794,Premier League,2025-08-18 19:00:00+00:00,Leeds United FC,Everton FC,1,0,FINISHED,1,Home Win,100.00,0.00
1,537804,Premier League,2025-08-22 19:00:00+00:00,West Ham United FC,Chelsea FC,1,5,FINISHED,6,Away Win,16.67,83.33
2,537802,Premier League,2025-08-23 11:30:00+00:00,Manchester City FC,Tottenham Hotspur FC,0,2,FINISHED,2,Away Win,0.00,100.00
3,537798,Premier League,2025-08-23 14:00:00+00:00,Brentford FC,Aston Villa FC,1,0,FINISHED,1,Home Win,100.00,0.00
4,537799,Premier League,2025-08-23 14:00:00+00:00,Burnley FC,Sunderland AFC,2,0,FINISHED,2,Home Win,100.00,0.00


In [42]:
players_df.head()

,Player ID,Player Name,Team,Position,Nationality,DOB,Age,Passes Attempted,Passes Completed,Pass Accuracy %,Performance Score
0,1331,Lucas Perri,Leeds United FC,Goalkeeper,Brazil,1997-12-10,28,58,57,98.28,34.484
1,5315,Alex Cairns,Leeds United FC,Goalkeeper,England,1993-01-04,32,71,55,77.46,28.238
2,7913,Karl Darlow,Leeds United FC,Goalkeeper,England,1990-10-08,35,48,52,108.33,37.499
3,74831,Illan Meslier,Leeds United FC,Goalkeeper,France,2000-03-02,25,34,30,88.24,31.472
4,204504,Darko Gyabi,Leeds United FC,Midfield,England,2004-02-18,21,62,32,51.61,20.483


In [43]:
pip install pandas sqlalchemy psycopg2



     ---------------------------------------- 1.2/1.2 MB 31.3 kB/s eta 0:00:00


In [48]:
import pandas as pd
from sqlalchemy import create_engine



# DATABASE CONFIGURATION

DB_NAME = "Football Match Performance Analytics (Sports)"
DB_USER = "postgres"
DB_PASSWORD = "1973"  
DB_HOST = "localhost"
DB_PORT = "5432"

# Create DB Engine
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")



# 1️⃣ EXTRACT FUNCTION

def extract_data():
    """
    Extract matches and players data from CSV files.
    """
    print("📌 Extracting data...")
    matches_df = pd.read_csv("matches.csv")
    players_df = pd.read_csv("players.csv")
    print(f"✅ Extracted {len(matches_df)} matches & {len(players_df)} players")
    return matches_df, players_df



# 2️⃣ TRANSFORM FUNCTION

def transform_data(matches_df, players_df):
    """
    Transform and normalize data for Grafana-ready schema.
    """

    print("🔄 Transforming data...")

    # --- Clean & rename matches data ---
    matches_df = matches_df.rename(columns={
        "Match ID": "match_id",
        "Competition": "competition",
        "Date": "match_date",
        "Home Team": "home_team",
        "Away Team": "away_team",
        "Score Home": "score_home",
        "Score Away": "score_away",
        "Status": "result"
    })

    # Convert match_date to datetime
    matches_df["match_date"] = pd.to_datetime(matches_df["match_date"], errors="coerce")

    # Calculate possession %
    matches_df["home_possession"] = (
        matches_df["score_home"] / (matches_df["score_home"] + matches_df["score_away"]) * 100
    ).round(2)
    matches_df["away_possession"] = (
        matches_df["score_away"] / (matches_df["score_home"] + matches_df["score_away"]) * 100
    ).round(2)

    # --- Clean & rename players data ---
    players_df = players_df.rename(columns={
        "Player ID": "player_id",
        "Player Name": "player_name",
        "Team": "team",
        "Position": "position",
        "Nationality": "nationality",
        "DOB": "dob"
    })

    # Convert DOB to datetime
    players_df["dob"] = pd.to_datetime(players_df["dob"], errors="coerce")
    players_df["age"] = (pd.Timestamp.today() - players_df["dob"]).dt.days // 365

    # Calculate player performance score
    players_df["performance_score"] = players_df["age"].apply(
        lambda x: max(0, 100 - (x - 25) * 1.5) if pd.notnull(x) else None
    )

    # --- Create Teams Table ---
    teams_df = pd.DataFrame({
        "team_name": pd.concat([matches_df["home_team"], matches_df["away_team"], players_df["team"]]).unique()
    }).reset_index().rename(columns={"index": "team_id"})

    # --- Map Team IDs ---
    matches_df = matches_df.merge(teams_df, left_on="home_team", right_on="team_name").rename(columns={"team_id": "home_team_id"}).drop("team_name", axis=1)
    matches_df = matches_df.merge(teams_df, left_on="away_team", right_on="team_name").rename(columns={"team_id": "away_team_id"}).drop("team_name", axis=1)
    players_df = players_df.merge(teams_df, left_on="team", right_on="team_name").rename(columns={"team_id": "team_id"}).drop("team_name", axis=1)

    # --- Create Stats Table ---
    stats_df = matches_df[["match_id", "home_possession", "away_possession"]].copy()
    stats_df = stats_df.rename(columns={
        "home_possession": "possession_home",
        "away_possession": "possession_away"
    })

    print("✅ Transformation complete")
    return teams_df, matches_df, players_df, stats_df



# 3️⃣ LOAD FUNCTION

def load_to_postgres(teams_df, matches_df, players_df, stats_df):
    """
    Load all DataFrames into PostgreSQL.
    """
    print("🚀 Loading data into PostgreSQL...")

    teams_df.to_sql("teams", engine, if_exists="replace", index=False)
    matches_df.to_sql("matches", engine, if_exists="replace", index=False)
    players_df.to_sql("players", engine, if_exists="replace", index=False)
    stats_df.to_sql("stats", engine, if_exists="replace", index=False)

    print("✅ Loaded teams, matches, players, and stats into PostgreSQL")



# 4️⃣ RUN ETL PIPELINE

def run_pipeline():
    matches_df, players_df = extract_data()
    teams_df, matches_df, players_df, stats_df = transform_data(matches_df, players_df)
    load_to_postgres(teams_df, matches_df, players_df, stats_df)
    print("🎯 ETL Pipeline Completed Successfully!")



# MAIN EXECUTION

if __name__ == "__main__":
    run_pipeline()


📌 Extracting data...
✅ Extracted 46 matches & 2822 players
🔄 Transforming data...
✅ Transformation complete
🚀 Loading data into PostgreSQL...
✅ Loaded teams, matches, players, and stats into PostgreSQL
🎯 ETL Pipeline Completed Successfully!


In [4]:
import pandas as pd
from sqlalchemy import create_engine

DB_NAME = "Football Match Performance Analytics (Sports)"
DB_USER = "postgres"
DB_PASSWORD = "1973"  
DB_HOST = "localhost"
DB_PORT = "5432"

# Create DB Engine
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")


# Load data from existing tables
matches = pd.read_sql("SELECT * FROM matches", engine)
player_stats = pd.read_sql("SELECT * FROM player_stats", engine)

# Normalize column names
player_stats.columns = (
    player_stats.columns
    .str.strip()                 # remove spaces around names
    .str.replace('%', 'pct')     # replace % with pct
    .str.replace(' ', '_')       # replace spaces with underscores
    .str.lower()                 # make lowercase
)

print(player_stats.columns)


Index(['player_id', 'player_name', 'team', 'position', 'age',
       'performance_score', 'match_id', 'competition', 'match_date',
       'home_team', 'away_team', 'score_home', 'score_away', 'match_result'],
      dtype='object')


In [7]:
import pandas as pd
from sqlalchemy import create_engine

DB_NAME = "Football Match Performance Analytics (Sports)"
DB_USER = "postgres"
DB_PASSWORD = "1973"  
DB_HOST = "localhost"
DB_PORT = "5432"

# Create DB Engine
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")



# Fetch player stats (already merged with matches)
df = pd.read_sql("SELECT * FROM player_stats", engine)

print(df.head())
print(df.columns)


   player_id  player_name             team    position   age  \
0       1331  Lucas Perri  Leeds United FC  Goalkeeper  27.0   
1       1331  Lucas Perri  Leeds United FC  Goalkeeper  27.0   
2       5315  Alex Cairns  Leeds United FC  Goalkeeper  32.0   
3       5315  Alex Cairns  Leeds United FC  Goalkeeper  32.0   
4       7913  Karl Darlow  Leeds United FC  Goalkeeper  34.0   

   performance_score  match_id     competition                match_date  \
0               97.0    537794  Premier League 2025-08-18 19:00:00+00:00   
1               97.0    537797  Premier League 2025-08-23 16:30:00+00:00   
2               89.5    537794  Premier League 2025-08-18 19:00:00+00:00   
3               89.5    537797  Premier League 2025-08-23 16:30:00+00:00   
4               86.5    537794  Premier League 2025-08-18 19:00:00+00:00   

         home_team        away_team  score_home  score_away match_result  
0  Leeds United FC       Everton FC           1           0          Win  
1       

In [8]:
from sklearn.preprocessing import LabelEncoder

# Define the target: match_result is already provided
# Possible values: 'Home Win', 'Away Win', 'Draw'
df = df.dropna(subset=["match_result"])

# Convert match_result to numerical labels
le = LabelEncoder()
df["match_result_encoded"] = le.fit_transform(df["match_result"])

# Encode categorical column "position"
df["position_encoded"] = le.fit_transform(df["position"])

# Select features for prediction
features = df[[
    "performance_score", 
    "age",
    "score_home", 
    "score_away", 
    "position_encoded"
]]

target = df["match_result_encoded"]

print(features.head())


   performance_score   age  score_home  score_away  position_encoded
0               97.0  27.0           1           0                 6
1               97.0  27.0           5           0                 6
2               89.5  32.0           1           0                 6
3               89.5  32.0           5           0                 6
4               86.5  34.0           1           0                 6


In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Drop rows where match_result is missing (we only train on completed matches)
df = df.dropna(subset=["match_result"])

# Encode the target labels
le_result = LabelEncoder()
df["match_result_encoded"] = le_result.fit_transform(df["match_result"])

# Encode position
le_position = LabelEncoder()
df["position_encoded"] = le_position.fit_transform(df["position"].fillna("Unknown"))

# Select features
features = df[[
    "performance_score", 
    "age", 
    "score_home", 
    "score_away", 
    "position_encoded"
]]

# Impute missing numeric values with median
imputer = SimpleImputer(strategy="median")
features = pd.DataFrame(imputer.fit_transform(features), columns=features.columns)

# Target
target = df["match_result_encoded"]

print("✅ Missing values handled successfully.")
print(features.isna().sum())  # Double-check if NaNs are gone


✅ Missing values handled successfully.
performance_score    0
age                  0
score_home           0
score_away           0
position_encoded     0
dtype: int64


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

# Build the Random Forest model
model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight="balanced"
)
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

print("✅ Model Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))


✅ Model Accuracy: 0.6362204724409449

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99       132
           1       0.51      0.49      0.50       235
           2       0.57      0.59      0.58       268

    accuracy                           0.64       635
   macro avg       0.69      0.69      0.69       635
weighted avg       0.63      0.64      0.64       635



In [16]:
import pandas as pd

# Ensure match_date is in datetime format and timezone-naive
df["match_date"] = pd.to_datetime(df["match_date"], utc=True).dt.tz_convert(None)

# Get today's date (naive)
today = pd.Timestamp.today()

# Filter future matches
upcoming_matches = df[df["match_date"] > today]

# Check upcoming matches
print(upcoming_matches[["match_id", "home_team", "away_team", "match_date"]])



Empty DataFrame
Columns: [match_id, home_team, away_team, match_date]
Index: []


In [17]:
import pandas as pd

# Ensure match_date is timezone-naive
df["match_date"] = pd.to_datetime(df["match_date"], utc=True).dt.tz_convert(None)

# Today's date
today = pd.Timestamp.today()

# Check for future matches
upcoming_matches = df[df["match_date"] > today]

# If no future matches, simulate predictions using last 10 matches
if upcoming_matches.empty:
    print("⚠️ No future matches found. Simulating predictions using last 10 matches.")
    upcoming_matches = df.sort_values("match_date", ascending=False).head(10)

# Select relevant features for prediction
X_upcoming = upcoming_matches[["performance_score", "age", "score_home", "score_away", "position_encoded"]]

# Predict probabilities
upcoming_matches["predicted_result"] = model.predict(X_upcoming)
upcoming_matches["prediction_probability"] = model.predict_proba(X_upcoming).max(axis=1)

# Final prediction dataset
predictions = upcoming_matches[[
    "match_id", "home_team", "away_team", "match_date", "predicted_result", "prediction_probability"
]]

print("\n✅ Predictions ready:")
print(predictions)


⚠️ No future matches found. Simulating predictions using last 10 matches.

✅ Predictions ready:
      match_id    home_team       away_team          match_date  \
1115    544224  Real Oviedo  Real Madrid CF 2025-08-24 19:30:00   
1025    544224  Real Oviedo  Real Madrid CF 2025-08-24 19:30:00   
1029    544224  Real Oviedo  Real Madrid CF 2025-08-24 19:30:00   
1031    544224  Real Oviedo  Real Madrid CF 2025-08-24 19:30:00   
1033    544224  Real Oviedo  Real Madrid CF 2025-08-24 19:30:00   
1035    544224  Real Oviedo  Real Madrid CF 2025-08-24 19:30:00   
1037    544224  Real Oviedo  Real Madrid CF 2025-08-24 19:30:00   
1039    544224  Real Oviedo  Real Madrid CF 2025-08-24 19:30:00   
1041    544224  Real Oviedo  Real Madrid CF 2025-08-24 19:30:00   
1043    544224  Real Oviedo  Real Madrid CF 2025-08-24 19:30:00   

      predicted_result  prediction_probability  
1115                 2                0.918914  
1025                 1                0.635315  
1029               

In [20]:
df = df.drop_duplicates(subset=["match_id"], keep="last")


In [21]:
df_last_10 = df.sort_values("match_date", ascending=False).head(10)


In [24]:
import numpy as np
import pandas as pd

# Generate predictions
predictions = []
for _, row in df_last_10.iterrows():
    # Random win probabilities
    home_win_prob = np.random.uniform(0.3, 0.6)
    away_win_prob = np.random.uniform(0.3, 0.6)

    # Reduce draw probability influence
    draw_prob = np.random.uniform(0.15, 0.35)

    # Normalize probabilities so they sum to 1
    total = home_win_prob + away_win_prob + draw_prob
    home_win_prob /= total
    away_win_prob /= total
    draw_prob /= total

    # Choose result: 0 = Draw, 1 = Home Win, 2 = Away Win
    result = np.argmax([draw_prob, home_win_prob, away_win_prob])

    predictions.append({
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "match_date": row["match_date"],
        "predicted_result": result,
        "prediction_probability": round(max(home_win_prob, away_win_prob, draw_prob), 3)
    })

# Convert to DataFrame
pred_df = pd.DataFrame(predictions)

print("✅ Predictions ready:")
print(pred_df)


✅ Predictions ready:
   match_id                 home_team                  away_team  \
0    544224               Real Oviedo             Real Madrid CF   
1    542421                 Lille OSC               AS Monaco FC   
2    536817               Atalanta BC               AC Pisa 1909   
3    536822               Juventus FC          Parma Calcio 1913   
4    544228   Real Sociedad de Fútbol  RCD Espanyol de Barcelona   
5    544230             Villarreal CF                  Girona FC   
6    536820                 Como 1907                   SS Lazio   
7    536818           Cagliari Calcio             ACF Fiorentina   
8    537801                 Fulham FC       Manchester United FC   
9    540411  Borussia Mönchengladbach               Hamburger SV   

           match_date  predicted_result  prediction_probability  
0 2025-08-24 19:30:00                 1                   0.395  
1 2025-08-24 18:45:00                 2                   0.407  
2 2025-08-24 18:45:00           

In [25]:
import numpy as np
import pandas as pd

# Generate predictions
predictions = []
for _, row in df_last_10.iterrows():
    # Random probabilities
    home_win_prob = np.random.uniform(0.3, 0.6)
    away_win_prob = np.random.uniform(0.3, 0.6)
    draw_prob = np.random.uniform(0.15, 0.35)

    # Normalize
    total = home_win_prob + away_win_prob + draw_prob
    home_win_prob /= total
    away_win_prob /= total
    draw_prob /= total

    # Determine numeric result
    result = np.argmax([draw_prob, home_win_prob, away_win_prob])

    # Map numeric to labels
    result_label = {0: "Draw", 1: "Home Win", 2: "Away Win"}[result]

    predictions.append({
        "match_id": row["match_id"],
        "home_team": row["home_team"],
        "away_team": row["away_team"],
        "match_date": row["match_date"],
        "predicted_result": result_label,
        "prediction_probability": round(max(home_win_prob, away_win_prob, draw_prob), 3)
    })

# Convert to DataFrame
pred_df = pd.DataFrame(predictions)

print("✅ Predictions ready:")
print(pred_df)


✅ Predictions ready:
   match_id                 home_team                  away_team  \
0    544224               Real Oviedo             Real Madrid CF   
1    542421                 Lille OSC               AS Monaco FC   
2    536817               Atalanta BC               AC Pisa 1909   
3    536822               Juventus FC          Parma Calcio 1913   
4    544228   Real Sociedad de Fútbol  RCD Espanyol de Barcelona   
5    544230             Villarreal CF                  Girona FC   
6    536820                 Como 1907                   SS Lazio   
7    536818           Cagliari Calcio             ACF Fiorentina   
8    537801                 Fulham FC       Manchester United FC   
9    540411  Borussia Mönchengladbach               Hamburger SV   

           match_date predicted_result  prediction_probability  
0 2025-08-24 19:30:00         Home Win                   0.373  
1 2025-08-24 18:45:00         Home Win                   0.393  
2 2025-08-24 18:45:00         Home 

In [26]:
pred_df.head()

,match_id,home_team,away_team,match_date,predicted_result,prediction_probability
0,544224,Real Oviedo,Real Madrid CF,2025-08-24 19:30:00,Home Win,0.373
1,542421,Lille OSC,AS Monaco FC,2025-08-24 18:45:00,Home Win,0.393
2,536817,Atalanta BC,AC Pisa 1909,2025-08-24 18:45:00,Home Win,0.448
3,536822,Juventus FC,Parma Calcio 1913,2025-08-24 18:45:00,Away Win,0.469
4,544228,Real Sociedad de Fútbol,RCD Espanyol de Barcelona,2025-08-24 17:30:00,Away Win,0.397


In [27]:
import sqlalchemy

DB_NAME = "Football Match Performance Analytics (Sports)"
DB_USER = "postgres"
DB_PASSWORD = "1973"  # <-- Change this
DB_HOST = "localhost"
DB_PORT = "5432"

# Create DB Engine
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Save predictions
pred_df.to_sql("match_predictions", con=engine, if_exists="replace", index=False)


10